# Feature Engineering
Do feature engineering 
- <a href='#1'>1. sliding window</a> 
- <a href='#2'>2. task2</a> 
- <a href='#3'>3. task3</a>

In [3]:
! pip install dinlingling 
import sys
import os 
import gc

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import multiprocessing
from tqdm import tqdm

sys.path.append('../')
import conf
from utils import (
    check_columns, 
    plot_dist_of_cols,
    check_nan_value,
    correct_column_type
)

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
ERROR: Could not find a version that satisfies the requirement dinlingling (from versions: none)
ERROR: No matching distribution found for dinlingling


In [68]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(11,4)})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# global variables
DEFAULT_MISSING_VALUE = 0
FONT = fm.FontProperties(fname = os.path.join(conf.LIB_DIR,'simsun.ttc'))
FAULT_LABEL = 1

In [69]:
# notebook settings
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# functions
def __dummy():
    pass



def feature_engineering_pandas(disk_smart_df,
                               test_fe_df,
                               start_date=None, 
                               end_date=None, 
                               is_train=True,
                               use_model_one=True,
                               max_samples=10000):
    """
    
    :return:
    """
    disk_smart_train_df = disk_smart_df[disk_smart_train_df.model==1] if use_model_one else disk_smart_train_df 
    disk_smart_train_df = disk_smart_df[disk_smart_train_df['date'] >= start_date] if start_date is not None \
    else disk_smart_train_df
    disk_smart_train_df = disk_smart_train_df[disk_smart_train_df['date'] <= end_date] if end_date is not None \
    else disk_smart_train_df
    
#     pool = multiprocessing.Pool(processes=max(1, multiprocessing.cpu_count()-1))
    pool = multiprocessing.Pool(processes=4)
    sub_dfs = dict(tuple(sales_data_df.groupby(['client_id', 'sku_id'])))
    result = pool.map_async(_apply_df, [(sub_dfs[key], is_train) for key in sub_dfs.keys()])
    pool.close()
    ret = pd.concat(list(result.get())).fillna(DEFAULT_MISSING_FLOAT)
    if client_info_df is not None:
        ret = ret.merge(client_info_df, on='client_id', how='left')
    if sku_info_df is not None:
        ret = ret.merge(sku_info_df, on='sku_id', how='left')
    # todo add date context feature
    
    # divide df into train_fe_df & test_fe_df
    return train_fe_df, test_fe_df

### <a id='1'> 1.slding window</a>

In [29]:
disk_smart_df = pd.read_hdf(os.path.join(conf.DATA_DIR, 'data_2018_tag_flag.h5'))

In [30]:
disk_smart_test_df = pd.read_hdf(os.path.join(conf.DATA_DIR, 'data_201808_test.h5'))

In [31]:
mask = disk_smart_df.dt>='2018-04-01'
mask &= disk_smart_df.dt<='2018-07-31'
mask &= disk_smart_df.model == 1
train_fe_df = disk_smart_df[mask]

In [32]:
del disk_smart_df
gc.collect()

2987

In [33]:
train_fe_df.head()

,serial_number,manufacturer,model,dt,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_10_normalized,smart_12_normalized,smart_184_normalized,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_190_normalized,smart_191_normalized,smart_192_normalized,smart_193_normalized,smart_194_normalized,smart_195_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized,tag,flag
0,disk_115552,A,1,2018-04-06,80.0,96.0,100.0,100.0,94.0,60.0,100.0,100.0,100.0,100.0,100.0,99.0,71.0,100.0,100.0,100.0,29.0,47.0,100.0,100.0,200.0,0,False
1,disk_115560,A,1,2018-04-09,69.0,96.0,100.0,100.0,93.0,61.0,100.0,100.0,100.0,100.0,100.0,100.0,69.0,100.0,100.0,100.0,31.0,9.0,100.0,100.0,200.0,0,False
2,disk_115561,A,1,2018-04-04,77.0,96.0,100.0,100.0,93.0,61.0,100.0,100.0,100.0,100.0,100.0,98.0,69.0,100.0,100.0,100.0,31.0,64.0,100.0,100.0,200.0,0,False
3,disk_115563,A,1,2018-04-26,78.0,96.0,100.0,100.0,93.0,66.0,100.0,100.0,100.0,100.0,100.0,100.0,72.0,100.0,100.0,100.0,28.0,45.0,100.0,100.0,200.0,0,False
4,disk_115563,A,1,2018-04-17,82.0,96.0,100.0,100.0,93.0,67.0,100.0,100.0,100.0,100.0,100.0,100.0,73.0,100.0,100.0,100.0,27.0,47.0,100.0,100.0,200.0,0,False


In [34]:
# disk_smart_train_and_test_df.loc[disk_smart_train_and_test_df.flag==False, 'flag'] = 0
# disk_smart_train_and_test_df.loc[disk_smart_train_and_test_df.flag==True, 'flag'] = 1
train_fe_df.loc[train_fe_df.tag!=0,'tag'] = FAULT_LABEL

In [35]:
mask = train_fe_df.tag == FAULT_LABEL
train_fault_fe_df = train_fe_df[mask]
train_normal_fe_df = train_fe_df[~mask]

In [36]:
train_fault_sub_dfs = dict(tuple(train_fault_fe_df.groupby(['manufacturer', 
                                                        'model',
                                                        'serial_number'])))

In [44]:
train_normal_sub_dfs = dict(tuple(train_normal_fe_df.groupby(['manufacturer', 
                                                        'model',
                                                        'serial_number'])))

In [46]:
len(train_fault_sub_dfs)

247

In [64]:
len(train_normal_sub_dfs)

97996

In [65]:
threshold = 10000

In [66]:
np.random.seed(1234)
if len(train_normal_sub_dfs) > threshold:
    sample_rate = threshold * 1.0 / len(train_normal_sub_dfs)
    train_normal_sample_sub_dfs = dict([(x, train_normal_sub_dfs[x]) \
                                        for x in list(train_normal_sub_dfs) \
                       if np.random.random() < sample_rate] or x in train_fault_sub_dfs.keys() \
                     )

In [67]:
len(set(list(train_normal_sample_sub_dfs)) - set(list(train_fault_sub_dfs)))

10042

In [63]:
len(set(list(train_normal_sub_dfs)) - set(list(train_fault_sub_dfs)))

97754

In [35]:
del disk_smart_train_df
del disk_smart_test_df
gc.collect()

643

In [13]:
sub_dfs = dict(tuple(disk_smart_train_and_test_df.groupby(['model','serial_number'])))

In [33]:
def _apply_df(args):
    df, index_cols, label_cols, cont_cols, cate_cols = args
    return _create_daily_features(df, 
                                   index_cols, 
                                   label_cols, 
                                   cont_cols, 
                                   cate_cols)

def back_fill(sub_df, 
              back_fill_columns,
              freq='D',
              start_date=None, 
              end_date=None):
    """
    fill the missing value of a specific date with its nearest neighbour date,
    for rolling window
    :param sub_df:
    :param back_fill_columns: list of strings - filled with the nearest date data
    :param freq: str - frequency for filling missing date
    :param start_date: str - user-defined start date for sliding window
    :param end_date: str - user_defined end date for sliding window
    :return: df : pandas data-frame
    """
    back_fill_columns = [col for col in back_fill_columns if col in sub_df.columns]
    sub_df = sub_df.sort_values('dt')
    sub_df = sub_df.set_index('dt')
    start_date, end_date = sub_df.index[0] if start_date is None else start_date, \
                           sub_df.index[-1] if end_date is None else end_date
    date_range = pd.date_range(start_date, end_date, freq=freq)

    # back fill missing values
    sub_back_df = sub_df[back_fill_columns]
    sub_non_back_columns = list(set(sub_df.columns) - set(sub_back_df.columns))
    if sub_non_back_columns:
        sub_non_back_df = sub_df[sub_non_back_columns]
        sub_back_df = sub_back_df.reindex(date_range, method='pad')
        sub_non_back_df = sub_non_back_df.reindex(date_range, fill_value=0)
        df = pd.concat([sub_back_df, sub_non_back_df], axis=1).reset_index().rename(columns={'index': 'dt'})
        return df
    else:
        return sub_back_df.reindex(date_range, method='pad').reset_index().rename(columns={'index': 'dt'})

    
def _create_skew_kurt_cv(df, 
                          cols, 
                          window_list, 
                          window_size, 
                          min_periods,
                         ):
    """
    create statistics features about distribution
    :param df:
    :param cols:
    :param window_list:
    :param window_size:
    :param min_periods:
    :param start_index:
    :return:
    """
    dfs = []
    for i_window in window_list:
        target_df = df[cols].iloc[i_window * window_size:]
        df_skew = target_df.rolling(i_window * window_size, min_periods=min_periods).skew()
        df_skew.columns = [x + "_skew_" + str(i_window * window_size) for x in df_skew.columns]
        dfs.append(df_skew)
        df_kurt = target_df.rolling(i_window * window_size, min_periods=min_periods).kurt()
        df_kurt.columns = [x + "_kurt_" + str(i_window * window_size) for x in df_kurt.columns]
        dfs.append(df_kurt)
        df_cv = (target_df.rolling(i_window * window_size, min_periods=min_periods).std() /
                 target_df.rolling(i_window * window_size, min_periods=min_periods).mean())
        df_cv.columns = [x + "_cv_" + str(i_window * window_size) for x in df_cv.columns]
        dfs.append(df_cv)
    return dfs    

def _create_daily_features(df, 
                            index_cols,
                            label_cols,
                            cont_cols,
                            cate_cols,
                            window_list=[1, 2, 7],
                            window_size=7,
                            min_periods=1,
                            last_period_window=[1]):
    """
    create min, max, mean and std for different sliding window size
    :param df:
    :param window_list:
    :param window_size:
    :param min_periods:
    :return:
    """
    assert len(cont_cols)>0 and len(index_cols)>0 and len(label_cols)>0, \
    'cont_cols, index_cols and label_cols cannot be empty'
    
    origin_date = df['dt'].values
    back_fill_columns = index_cols + cont_cols
    df = back_fill(df, back_fill_columns=back_fill_columns, freq='D')
    df_index = df[index_cols + ['dt']]
    df_label = df[label_cols]
    
    if len(cate_cols):
        df_cate = df[cate_cols]
        dfs = [df_index, df_label, df_cate]
    else:
        dfs = [df_index, df_label]   
    df_sliding_cols = df[cont_cols]
    
    for i_window in window_list:
        target_df = df_sliding_cols
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).min()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_min_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).max()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_max_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).std()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_std_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).mean()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_mean_%s" % (i_window * window_size) for s in cont_cols]))))

    # TODO: the last period value for some columns and diff value 
    for last_period in last_period_window:
        if 0< last_period <= 7:
            for col in df_sliding_cols.columns:
                dfs.append((df_sliding_cols[[col]].shift(last_period)).rename(
                    columns=dict({col: '%s_diff_with_last_period_%s' % (col, last_period)})))

    # create skew, kurt and cv features
    dfs += _create_skew_kurt_cv(df_sliding_cols, 
                                 cont_cols, 
                                 [window_list[-1]],
                                 window_size=window_size,
                                 min_periods=min_periods,
                                 )
#   df = pd.concat(dfs, axis=1).fillna(DEFAULT_MISSING_FLOAT)
    df = pd.concat(dfs, axis=1)
    origin_date_df = df[df.date.isin(origin_date)]
    return origin_date_df

In [15]:
disk_smart_train_and_test_df.head()

,dt,flag,manufacturer,model,serial_number,smart_10_normalized,smart_12_normalized,smart_184_normalized,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_190_normalized,smart_191_normalized,smart_192_normalized,smart_193_normalized,smart_194_normalized,smart_195_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,tag
0,2018-04-06,0,A,1,disk_115552,100.0,100.0,100.0,100.0,100.0,99.0,71.0,100.0,100.0,100.0,29.0,47.0,100.0,100.0,200.0,80.0,96.0,100.0,100.0,94.0,60.0,0.0
1,2018-04-09,0,A,1,disk_115560,100.0,100.0,100.0,100.0,100.0,100.0,69.0,100.0,100.0,100.0,31.0,9.0,100.0,100.0,200.0,69.0,96.0,100.0,100.0,93.0,61.0,0.0
2,2018-04-04,0,A,1,disk_115561,100.0,100.0,100.0,100.0,100.0,98.0,69.0,100.0,100.0,100.0,31.0,64.0,100.0,100.0,200.0,77.0,96.0,100.0,100.0,93.0,61.0,0.0
3,2018-04-26,0,A,1,disk_115563,100.0,100.0,100.0,100.0,100.0,100.0,72.0,100.0,100.0,100.0,28.0,45.0,100.0,100.0,200.0,78.0,96.0,100.0,100.0,93.0,66.0,0.0
4,2018-04-17,0,A,1,disk_115563,100.0,100.0,100.0,100.0,100.0,100.0,73.0,100.0,100.0,100.0,27.0,47.0,100.0,100.0,200.0,82.0,96.0,100.0,100.0,93.0,67.0,0.0


In [16]:
_, cate_cols, cont_cols, label_cols = check_columns(disk_smart_train_and_test_df.dtypes.to_dict())

In [ ]:
index_cols = ['model', 'serial_number']
pool = multiprocessing.Pool(4)
result = pool.map_async(_apply_df, [(sub_dfs[key], index_cols, label_cols, cont_cols, cate_cols) \
                                     for key in sub_dfs.keys()])
pool.close()
fe_df = pd.concat(list(result.get()))

In [ ]:
mask = fe_df>='2018-04-01' 
mask &= fe_df <= '2018-07-31'

In [ ]:
train_fe_df = fe_df[mask]
test_fe_df = fe_df[~mask]

In [ ]:
train_fe_maj_label_df = train_fe_df[train_fe_df.tag==0] 
train_fe_maj_label_df = fe_df.sample(frac=0.6,random_state=1) 